# Introduction
For this analysis, I will perform EDA on the Food.com recipes dataset. My primary goal will be classifying the recipes beased on time and ratings. It will be useful to people who wants to choose the recipes based on its nutrition and the amount of time required to cook. 

I will use the raw recipes and interactions datasets to perform the analysis.

# Understanding the Data

## Importing Libraries and DataSet

In [8]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer

import copy
import operator
import datetime
import warnings
from itertools import product
warnings.filterwarnings("ignore")


#useful way to visualize directory
print(os.path.join(os.getcwd(),"input"))
os.listdir(os.path.join(os.getcwd(), "input"))



C:\Users\arun\Documents\food-lover\input


KeyError: 'recipe_id'

In [15]:
recipes_RAW = pd.read_csv(os.path.join(os.getcwd(), "input","RAW_recipes.csv"))
recipes_RAW.sort_values(by=['id'])
recipes_RAW = recipes_RAW.rename({"id":"recipe_id"},axis=1)
#recipes_RAW.rename({"id":"recipe_id"},axis=1)
recipes_RAW.head(5)

#recipes_RAW = recipes_RAW.rename({"id":"recipe_id"},axis=1)
#interactions_RAW = pd.read_csv("../input/food-com-recipes-and-user-interactions/RAW_interactions.csv")


,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [13]:
interactions_RAW = pd.read_csv(os.path.join(os.getcwd(), "input","RAW_interactions.csv"))
interactions_RAW.sort_values(by=['recipe_id'])
interactions_RAW.head(5)

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [16]:
recipes_RAW.sort_values(by=['recipe_id'])
result = pd.merge(recipes_RAW,
                 interactions_RAW,
                 on='recipe_id')
result.head(5)

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,user_id,date,rating,review
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,4470,2006-02-18,5,I used an acorn squash and recipe#137681 Swee...
1,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,593927,2010-08-21,5,This was a nice change. I used butternut squas...
2,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,178427,2011-12-05,5,Excellent recipe! I used butternut squash and ...
3,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,28603,2002-08-19,0,"Have not tried this, but it sounds delicious. ..."
4,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,346277,2006-08-27,5,This recipe was wonderful. Instead of using t...


In [87]:
from string import digits
filtered_result = result[['name', 'recipe_id', 'minutes', 'rating', 'review']]
#filtered_result.drop_duplicates(['recipe_id'], inplace=True)
filtered_result.head(5)
#digits
#filtered_result['name'] = filtered_result['name'].str.lstrip(digits)
#filtered_result['name'].str.lstrip(digits).head(5)
#filtered_result['name'].head(5)

,name,recipe_id,minutes,rating,review
0,arriba baked winter squash mexican style,137739,55,5,I used an acorn squash and recipe#137681 Swee...
1,arriba baked winter squash mexican style,137739,55,5,This was a nice change. I used butternut squas...
2,arriba baked winter squash mexican style,137739,55,5,Excellent recipe! I used butternut squash and ...
3,a bit different breakfast pizza,31490,30,0,"Have not tried this, but it sounds delicious. ..."
4,a bit different breakfast pizza,31490,30,5,This recipe was wonderful. Instead of using t...


In [89]:
plt.style.use('ggplot')
# Draw Plot
plt.figure(figsize=(13,10), dpi= 80)
#sns.boxplot(x='class', y='hwy', data=filtered_result, notch=False)
#filtered_result.groupby('recipe_id', group_keys=False).apply(lambda x: x.loc[x.B.idavg()])
#filtered_result_rating = pd.pivot_table(filtered_result, columns = ['recipe_id'], index = ['name', 'minutes'],
#                    values = ['rating'], aggfunc = np.mean)
#filtered_result['count'] = filtered_result.groupby('recipe_id')['recipe_id'].transform('count')
filtered_result.groupby(['recipe_id', 'name', 'minutes',],group_keys=False).mean().reset_index()
filtered_result.sort_values(ascending=False, by=['rating'])

,name,recipe_id,minutes,rating,review,count
0,arriba baked winter squash mexican style,137739,55,5,I used an acorn squash and recipe#137681 Swee...,3
693162,most incredible no fail pie crust,28943,10,5,"You are right, this pie crust is both incredib...",177
693148,most incredible no fail pie crust,28943,10,5,Thank you so much for a great recipe. I have ...,177
693149,most incredible no fail pie crust,28943,10,5,I'm so excited. I have never been able to mak...,177
693150,most incredible no fail pie crust,28943,10,5,I just pulled up this recipe to make a pumpkin...,177
...,...,...,...,...,...,...
768874,pat s tomato bread soup,267157,80,0,I made the soup as directed and it was basical...,2
262332,cinnamon flat bread,156504,30,0,As I was enjoying this cinnamon bread (althoug...,5
768883,pat in pan crust,376807,40,0,Something is amiss with the flour quantities h...,1
768899,patacones fried plantain,92191,25,0,For a sweet treat try frying a little cinnamon...,9


<Figure size 1040x800 with 0 Axes>